In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib.ticker import FormatStrFormatter, MaxNLocator
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# ------------------------------
# USER INPUTS
# ------------------------------

pollutant_files = {
    'CO':    r"D:\IPMA\Results\co_fire_meteo_Iberia.nc",
    'NO':    r"D:\IPMA\Results\no_fire_meteo_Iberia.nc",
    'NO2':   r"D:\IPMA\Results\no2_fire_meteo_Iberia.nc",
    'PM2.5': r"D:\IPMA\Results\pm2p5_fire_meteo_Iberia.nc",
    'PM10':  r"D:\IPMA\Results\pm10_fire_meteo_Iberia.nc"
}

N_days = 25

meteo_vars = {
    "precip_Total_Precipitation": ("mm", "PuBu"),
    "temp_Max": ("°C", "coolwarm"),
    "wind_Max": ("m/s", "Oranges")
}

meteo_var_names = {
    "precip_Total_Precipitation": "Total Precipitation",
    "temp_Max": "Max Temperature",
    "wind_Max": "Max Wind Speed"
}

global_mean_output_file = r"D:\IPMA\Results\global_mean_meteo.nc" 
composite_output_dir = r"D:\IPMA\Results"

# --- Helper functions ---
def apply_unit_conversions(ds, pollutant_name):
    if pollutant_name == 'CO' and 'Mean' in ds:
        ds['Mean'] = ds['Mean'] * 1000
        if 'units' in ds['Mean'].attrs and ds['Mean'].attrs['units'] == 'mg/m3':
            ds['Mean'].attrs['units'] = 'microg/m3'
    
    precip_var = "precip_Total_Precipitation"
    if precip_var in ds:
        ds[precip_var] = ds[precip_var] * 1000
        if 'units' in ds[precip_var].attrs and ds[precip_var].attrs['units'] == 'm':
            ds[precip_var].attrs['units'] = 'mm'

    return ds

def set_composite_attrs(da, comp_type, var_name, pollutant, unit, N_days):
    da.name = f'{var_name}_{comp_type}'
    da.attrs['units'] = unit
    da.attrs['long_name'] = f'{comp_type} Composite of {meteo_var_names[var_name]} based on {pollutant} extremes'
    da.attrs['N_days'] = N_days
    return da

def calculate_dynamic_extent(data_array):
    mask = np.isfinite(data_array.values)
    if not mask.any(): return None

    lat_vals = data_array["latitude"].values
    lon_vals = data_array["longitude"].values

    valid_idx = np.where(mask)
    unique_lat_idx = np.unique(valid_idx[0])
    unique_lon_idx = np.unique(valid_idx[1])

    lat_step = lat_vals[1] - lat_vals[0] if len(lat_vals) > 1 else 0.5
    lon_step = lon_vals[1] - lon_vals[0] if len(lon_vals) > 1 else 0.5

    lat_min_ext = max(lat_vals[unique_lat_idx].min() - lat_step, lat_vals.min())
    lat_max_ext = min(lat_vals[unique_lat_idx].max() + lat_step, lat_vals.max())
    lon_min_ext = max(lon_vals[unique_lon_idx].min() - lon_step, lon_vals.min())
    lon_max_ext = min(lon_vals[unique_lon_idx].max() + lon_step, lon_vals.max())
    
    return [lon_min_ext, lon_max_ext, lat_min_ext, lat_max_ext]

# ------------------------------
# PART 1: GLOBAL MEAN CALCULATION
# ------------------------------

print("--- Part 1: Calculating & Saving Global Mean (Climatology) ---")
reference_pollutant = 'CO'
reference_file = pollutant_files[reference_pollutant]

try:
    ds_ref = xr.open_dataset(reference_file)
except FileNotFoundError:
    print(f"ERROR: Reference file not found at {reference_file}. Exiting.")
    exit()

ds_ref = apply_unit_conversions(ds_ref, reference_pollutant)
global_means = {}
for var_name, (unit, cmap) in meteo_vars.items():
    mean_da = ds_ref[var_name].mean(dim='time')
    mean_da.attrs['units'] = unit
    mean_da.attrs['long_name'] = f'Time-Mean of {meteo_var_names[var_name]}'
    global_means[var_name] = mean_da

ds_global_mean = xr.Dataset(global_means)
output_dir = os.path.dirname(global_mean_output_file)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
ds_global_mean.to_netcdf(global_mean_output_file, mode='w')
print(f"Global Mean data successfully saved to {global_mean_output_file}.")

# ------------------------------
# PART 2: COMPOSITING & ANOMALY CALCULATION
# ------------------------------

print("\n--- Part 2: Compositing Analysis and Saving Composites ---")
if not os.path.exists(composite_output_dir):
    os.makedirs(composite_output_dir)

all_composite_results = {}

for target_pollutant, target_file in pollutant_files.items():
    
    try:
        ds = xr.open_dataset(target_file)
    except FileNotFoundError:
        print(f"Skipping {target_pollutant}: File not found at {target_file}")
        continue
        
    ds = apply_unit_conversions(ds, target_pollutant)
        
    pollutant_time_series = ds['Mean'].mean(dim=['latitude', 'longitude'])
    ranked_series = pollutant_time_series.to_series().sort_values(ascending=False)
    high_days_time_stamps = ranked_series.index[:N_days]
    low_days_time_stamps = ranked_series.index[-N_days:]

    composite_data_vars = {} 
    composite_results_pollutant = {}

    for var_name, (unit, cmap) in meteo_vars.items():
        
        # Composites
        high_composite = ds[var_name].sel(time=high_days_time_stamps).mean(dim='time')
        low_composite = ds[var_name].sel(time=low_days_time_stamps).mean(dim='time')
        
        # Anomalies
        global_mean_data = global_means[var_name]
        high_anomaly = high_composite - global_mean_data 
        low_anomaly = low_composite - global_mean_data  

        # Store results for plotting
        composite_results_pollutant[var_name] = {
            'High_Comp': high_composite, 
            'Low_Comp': low_composite,   
            'High_Anomaly': high_anomaly,
            'Low_Anomaly': low_anomaly,
            'unit': unit,
            'cmap': cmap
        }

        # Store all generated fields for NetCDF saving
        composite_data_vars[f'{var_name}_High_Comp'] = set_composite_attrs(high_composite.copy(), 'High_Comp', var_name, target_pollutant, unit, N_days)
        composite_data_vars[f'{var_name}_Low_Comp'] = set_composite_attrs(low_composite.copy(), 'Low_Comp', var_name, target_pollutant, unit, N_days)
        composite_data_vars[f'{var_name}_High_Anom'] = set_composite_attrs(high_anomaly.copy(), 'High_Anom', var_name, target_pollutant, unit, N_days)
        composite_data_vars[f'{var_name}_Low_Anom'] = set_composite_attrs(low_anomaly.copy(), 'Low_Anom', var_name, target_pollutant, unit, N_days)

    all_composite_results[target_pollutant] = composite_results_pollutant
    
    # Save composites to NetCDF
    composite_file_name = f"{target_pollutant}_composite_meteo.nc"
    output_path = os.path.join(composite_output_dir, composite_file_name)
    ds_composite = xr.Dataset(
        composite_data_vars,
        coords={'latitude': high_composite['latitude'], 'longitude': high_composite['longitude']}
    )
    ds_composite.attrs['title'] = f'Composite Analysis for {target_pollutant} Extremes (N={N_days})'
    ds_composite.to_netcdf(output_path, mode='w')
    
print("Data processing complete. Starting visualizations.")

# ------------------------------
# PART 3: VISUALIZATION
# ------------------------------

print("\n--- Part 3: Generating Visualization Plots ---")
num_cols = len(meteo_vars)
meteo_list = list(meteo_vars.keys())

reference_data_for_extent = ds_global_mean[meteo_list[0]]
global_extent = calculate_dynamic_extent(reference_data_for_extent)
if global_extent is None:
    global_extent = [ds_global_mean.longitude.min().item(), ds_global_mean.longitude.max().item(), 
                     ds_global_mean.latitude.min().item(), ds_global_mean.latitude.max().item()]
    
TITLE_FONT_SIZE = 15
SUBTITLE_FONT_SIZE = 13
AXIS_FONT_SIZE = 11
CBAR_FONT_SIZE = 10

# --- Global Mean Climatologies ---
fig1, axes1 = plt.subplots(
    1, num_cols, figsize=(4 * num_cols, 3.5),
    subplot_kw={'projection': ccrs.PlateCarree()}
)
if num_cols == 1: axes1 = [axes1]

for i, var_name in enumerate(meteo_list):
    ax = axes1[i]
    mean_data = global_means[var_name]
    unit, cmap = meteo_vars[var_name]
    var_display_name = meteo_var_names[var_name]
    
    plot = mean_data.plot.pcolormesh(
        ax=ax, x='longitude', y='latitude', cmap=cmap, 
        transform=ccrs.PlateCarree(), add_colorbar=False, rasterized=True
    )
    ax.set_extent(global_extent, crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linewidth=0.8)
    ax.set_title(f"{var_display_name}", fontsize=SUBTITLE_FONT_SIZE) 
    ax.set_xlabel("Longitude", fontsize=AXIS_FONT_SIZE)
    ax.set_ylabel("Latitude", fontsize=AXIS_FONT_SIZE)
    
    cbar = plt.colorbar(plot, ax=ax, orientation='horizontal', pad=0.08, aspect=40,
                        label=f'({unit})', shrink=0.7)
    cbar.locator = MaxNLocator(nbins=8)
    cbar.update_ticks()
    cbar.ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    cbar.set_label(f"({unit})", fontsize=CBAR_FONT_SIZE)
    cbar.ax.tick_params(labelsize=CBAR_FONT_SIZE)

fig1.suptitle("Global Mean Climatology of Meteorological Variables", fontsize=TITLE_FONT_SIZE, y=1.02)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

# --- Raw Composites ---
for pollutant in pollutant_files.keys():
    if pollutant not in all_composite_results: continue
    for var_name in meteo_list:
        result = all_composite_results[pollutant][var_name]
        high_comp = result['High_Comp']
        low_comp = result['Low_Comp']
        unit, cmap = result['unit'], meteo_vars[var_name][1]
        var_display_name = meteo_var_names[var_name]
        
        max_val = np.nanmax([high_comp.values, low_comp.values])
        min_val = np.nanmin([high_comp.values, low_comp.values])
        
        fig, axes = plt.subplots(
            1, 2, figsize=(10, 4), subplot_kw={'projection': ccrs.PlateCarree()}
        )
        
        # High Composite
        ax_high = axes[0]
        plot_high = high_comp.plot.pcolormesh(
            ax=ax_high, x='longitude', y='latitude', cmap=cmap, 
            vmin=min_val, vmax=max_val, add_colorbar=False,
            transform=ccrs.PlateCarree(), rasterized=True
        )
        ax_high.set_extent(global_extent, crs=ccrs.PlateCarree())
        ax_high.coastlines()
        ax_high.add_feature(cfeature.BORDERS, linewidth=0.8)
        ax_high.set_title(f"High {pollutant} Composite", fontsize=SUBTITLE_FONT_SIZE)
        ax_high.set_xlabel("Longitude", fontsize=AXIS_FONT_SIZE)
        ax_high.set_ylabel("Latitude", fontsize=AXIS_FONT_SIZE)
        cbar_high = plt.colorbar(plot_high, ax=ax_high, orientation='horizontal', shrink=0.8, pad=0.05)
        cbar_high.set_label(f'({unit})', fontsize=CBAR_FONT_SIZE)
        
        # Low Composite
        ax_low = axes[1]
        plot_low = low_comp.plot.pcolormesh(
            ax=ax_low, x='longitude', y='latitude', cmap=cmap, 
            vmin=min_val, vmax=max_val, add_colorbar=False,
            transform=ccrs.PlateCarree(), rasterized=True
        )
        ax_low.set_extent(global_extent, crs=ccrs.PlateCarree())
        ax_low.coastlines()
        ax_low.add_feature(cfeature.BORDERS, linewidth=0.8)
        ax_low.set_title(f"Low {pollutant} Composite", fontsize=SUBTITLE_FONT_SIZE)
        ax_low.set_xlabel("Longitude", fontsize=AXIS_FONT_SIZE)
        ax_low.set_ylabel("")
        cbar_low = plt.colorbar(plot_low, ax=ax_low, orientation='horizontal', shrink=0.8, pad=0.05)
        cbar_low.set_label(f'({unit})', fontsize=CBAR_FONT_SIZE)

        fig.suptitle(f"{var_display_name} based on {pollutant} Extremes (n={N_days})", fontsize=TITLE_FONT_SIZE, y=1.02)
        plt.tight_layout(rect=[0, 0, 1, 0.98])
        plt.show()

# --- Composite Anomalies ---
for pollutant in pollutant_files.keys():
    if pollutant not in all_composite_results: continue
    for var_name in meteo_list:
        result = all_composite_results[pollutant][var_name]
        high_anomaly = result['High_Anomaly']
        low_anomaly = result['Low_Anomaly']
        unit = result['unit']
        var_display_name = meteo_var_names[var_name]
        
        max_abs = np.nanmax(np.abs([high_anomaly.values, low_anomaly.values])) if high_anomaly.values.size > 0 else 1e-6
        
        fig, axes = plt.subplots(
            1, 2, figsize=(10, 4), subplot_kw={'projection': ccrs.PlateCarree()}
        )
        
        # High Anomaly
        ax_high = axes[0]
        plot_high = high_anomaly.plot.pcolormesh(
            ax=ax_high, x='longitude', y='latitude', cmap='RdBu_r', 
            vmin=-max_abs, vmax=max_abs, add_colorbar=False,
            transform=ccrs.PlateCarree(), rasterized=True
        )
        ax_high.set_extent(global_extent, crs=ccrs.PlateCarree())
        ax_high.coastlines()
        ax_high.add_feature(cfeature.BORDERS, linewidth=0.8)
        ax_high.set_title(f"High {pollutant} Anomaly", fontsize=SUBTITLE_FONT_SIZE)
        ax_high.set_xlabel("Longitude", fontsize=AXIS_FONT_SIZE)
        ax_high.set_ylabel("Latitude", fontsize=AXIS_FONT_SIZE)
        cbar_high = plt.colorbar(plot_high, ax=ax_high, orientation='horizontal', shrink=0.8, pad=0.05)
        cbar_high.set_label(f'({unit})', fontsize=CBAR_FONT_SIZE)
        
        # Low Anomaly
        ax_low = axes[1]
        plot_low = low_anomaly.plot.pcolormesh(
            ax=ax_low, x='longitude', y='latitude', cmap='RdBu_r', 
            vmin=-max_abs, vmax=max_abs, add_colorbar=False,
            transform=ccrs.PlateCarree(), rasterized=True
        )
        ax_low.set_extent(global_extent, crs=ccrs.PlateCarree())
        ax_low.coastlines()
        ax_low.add_feature(cfeature.BORDERS, linewidth=0.8)
        ax_low.set_title(f"Low {pollutant} Anomaly", fontsize=SUBTITLE_FONT_SIZE)
        ax_low.set_xlabel("Longitude", fontsize=AXIS_FONT_SIZE)
        ax_low.set_ylabel("")
        cbar_low = plt.colorbar(plot_low, ax=ax_low, orientation='horizontal', shrink=0.8, pad=0.05)
        cbar_low.set_label(f'({unit})', fontsize=CBAR_FONT_SIZE)

        fig.suptitle(f"{var_display_name} based on {pollutant} Extremes (n={N_days})", fontsize=TITLE_FONT_SIZE, y=1.02)
        plt.tight_layout(rect=[0, 0, 1, 0.98])
        plt.show()

print("\nCode execution finished. Composites and Anomalies now plotted without difference fields.")


In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib.ticker import FormatStrFormatter, MaxNLocator
# Import Cartopy for geographical plotting
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# ------------------------------
# USER INPUTS (UNCHANGED)
# ------------------------------

pollutant_files = {
    'CO':    r"D:\IPMA\Results\co_fire_meteo_Iberia.nc",
    'NO':    r"D:\IPMA\Results\no_fire_meteo_Iberia.nc",
    'NO2':   r"D:\IPMA\Results\no2_fire_meteo_Iberia.nc",
    'PM2.5': r"D:\IPMA\Results\pm2p5_fire_meteo_Iberia.nc",
    'PM10':  r"D:\IPMA\Results\pm10_fire_meteo_Iberia.nc"
}

N_days = 25

meteo_vars = {
    "precip_Total_Precipitation": ("mm", "PuBu"),
    "temp_Max": ("°C", "coolwarm"),
    "wind_Max": ("m/s", "Oranges")
}

meteo_var_names = {
    "precip_Total_Precipitation": "Total Precipitation",
    "temp_Max": "Max Temperature",
    "wind_Max": "Max Wind Speed"
}

global_mean_output_file = r"D:\IPMA\Results\global_mean_meteo.nc" 
composite_output_dir = r"D:\IPMA\Results"


# --- Helper functions (UNCHANGED) ---
def apply_unit_conversions(ds, pollutant_name):
    """
    Applies necessary unit conversions (CO and Precipitation) to the xarray Dataset.
    """
    if pollutant_name == 'CO' and 'Mean' in ds:
        ds['Mean'] = ds['Mean'] * 1000
        if 'units' in ds['Mean'].attrs and ds['Mean'].attrs['units'] == 'mg/m3':
            ds['Mean'].attrs['units'] = 'microg/m3'
    
    precip_var = "precip_Total_Precipitation"
    if precip_var in ds:
        ds[precip_var] = ds[precip_var] * 1000
        if 'units' in ds[precip_var].attrs and ds[precip_var].attrs['units'] == 'm':
            ds[precip_var].attrs['units'] = 'mm'

    return ds

def set_composite_attrs(da, comp_type, var_name, pollutant, unit, N_days):
    """Helper function to set attributes for saving."""
    da.name = f'{var_name}_{comp_type}'
    da.attrs['units'] = unit
    da.attrs['long_name'] = f'{comp_type} Composite of {meteo_var_names[var_name]} based on {pollutant} extremes'
    da.attrs['N_days'] = N_days
    return da

# --- Zoom Calculation Helper Function (UNCHANGED) ---
def calculate_dynamic_extent(data_array):
    """Calculates the tightest bounding box around non-NaN data points."""
    mask = np.isfinite(data_array.values)
    
    if not mask.any():
        return None # No valid data

    lat_vals = data_array["latitude"].values
    lon_vals = data_array["longitude"].values

    valid_idx = np.where(mask)
    unique_lat_idx = np.unique(valid_idx[0])
    unique_lon_idx = np.unique(valid_idx[1])

    # Determine grid cell size for padding
    lat_step = lat_vals[1] - lat_vals[0] if len(lat_vals) > 1 else 0.5
    lon_step = lon_vals[1] - lon_vals[0] if len(lon_vals) > 1 else 0.5

    # Calculate extent with a small buffer (one grid step)
    lat_min_ext = max(lat_vals[unique_lat_idx].min() - lat_step, lat_vals.min())
    lat_max_ext = min(lat_vals[unique_lat_idx].max() + lat_step, lat_vals.max())
    lon_min_ext = max(lon_vals[unique_lon_idx].min() - lon_step, lon_vals.min())
    lon_max_ext = min(lon_vals[unique_lon_idx].max() + lon_step, lon_vals.max())
    
    return [lon_min_ext, lon_max_ext, lat_min_ext, lat_max_ext]


# ------------------------------
# PART 1: GLOBAL MEAN CALCULATION (UNCHANGED LOGIC)
# ------------------------------

print("--- Part 1: Calculating & Saving Global Mean (Climatology) ---")
reference_pollutant = 'CO'
reference_file = pollutant_files[reference_pollutant]

try:
    ds_ref = xr.open_dataset(reference_file)
except FileNotFoundError:
    print(f"ERROR: Reference file not found at {reference_file}. Exiting.")
    exit()

ds_ref = apply_unit_conversions(ds_ref, reference_pollutant)
global_means = {}
for var_name, (unit, cmap) in meteo_vars.items():
    mean_da = ds_ref[var_name].mean(dim='time')
    mean_da.attrs['units'] = unit
    mean_da.attrs['long_name'] = f'Time-Mean of {meteo_var_names[var_name]}'
    global_means[var_name] = mean_da

ds_global_mean = xr.Dataset(global_means)
output_dir = os.path.dirname(global_mean_output_file)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
ds_global_mean.to_netcdf(global_mean_output_file, mode='w')
print(f"Global Mean data successfully saved to {global_mean_output_file}.")

# ------------------------------
# PART 2: COMPOSITING & ANOMALY CALCULATION (UNCHANGED LOGIC)
# ------------------------------

print("\n--- Part 2: Compositing Analysis and Saving Composites ---")
if not os.path.exists(composite_output_dir):
    os.makedirs(composite_output_dir)

all_composite_results = {}

for target_pollutant, target_file in pollutant_files.items():
    
    try:
        ds = xr.open_dataset(target_file)
    except FileNotFoundError:
        print(f"Skipping {target_pollutant}: File not found at {target_file}")
        continue
        
    ds = apply_unit_conversions(ds, target_pollutant)
        
    pollutant_time_series = ds['Mean'].mean(dim=['latitude', 'longitude'])
    ranked_series = pollutant_time_series.to_series().sort_values(ascending=False)
    high_days_time_stamps = ranked_series.index[:N_days]
    low_days_time_stamps = ranked_series.index[-N_days:]

    composite_data_vars = {} 
    composite_results_pollutant = {}

    for var_name, (unit, cmap) in meteo_vars.items():
        
        # Composites (Raw Averages)
        high_composite = ds[var_name].sel(time=high_days_time_stamps).mean(dim='time')
        low_composite = ds[var_name].sel(time=low_days_time_stamps).mean(dim='time')
        
        # Anomalies
        global_mean_data = global_means[var_name]
        high_anomaly = high_composite - global_mean_data 
        low_anomaly = low_composite - global_mean_data  
        
        # Difference
        difference_anomaly = high_anomaly - low_anomaly

        # Store results for in-memory use (for plotting Part 3)
        composite_results_pollutant[var_name] = {
            'High_Comp': high_composite, 
            'Low_Comp': low_composite,   
            'High_Anomaly': high_anomaly,
            'Low_Anomaly': low_anomaly,
            'Difference': difference_anomaly,
            'unit': unit,
            'cmap': cmap
        }

        # Store all generated fields in the dictionary for NetCDF saving
        composite_data_vars[f'{var_name}_High_Comp'] = set_composite_attrs(high_composite.copy(), 'High_Comp', var_name, target_pollutant, unit, N_days)
        composite_data_vars[f'{var_name}_Low_Comp'] = set_composite_attrs(low_composite.copy(), 'Low_Comp', var_name, target_pollutant, unit, N_days)
        composite_data_vars[f'{var_name}_High_Anom'] = set_composite_attrs(high_anomaly.copy(), 'High_Anom', var_name, target_pollutant, unit, N_days)
        composite_data_vars[f'{var_name}_Low_Anom'] = set_composite_attrs(low_anomaly.copy(), 'Low_Anom', var_name, target_pollutant, unit, N_days)
        composite_data_vars[f'{var_name}_Diff'] = set_composite_attrs(difference_anomaly.copy(), 'Diff', var_name, target_pollutant, unit, N_days)

    all_composite_results[target_pollutant] = composite_results_pollutant
    
    # 2.4. SAVE THE COMPOSITES TO NETCDF
    composite_file_name = f"{target_pollutant}_composite_meteo.nc"
    output_path = os.path.join(composite_output_dir, composite_file_name)
    ds_composite = xr.Dataset(
        composite_data_vars,
        coords={'latitude': high_composite['latitude'], 'longitude': high_composite['longitude']}
    )
    ds_composite.attrs['title'] = f'Composite Analysis for {target_pollutant} Extremes (N={N_days})'
    ds_composite.to_netcdf(output_path, mode='w')
    
print("Data processing complete. Starting visualizations.")


# ----------------------------------------------------------------------
# PART 3: VISUALIZATION - IMPLEMENTING DEDICATED COLORBARS
# ----------------------------------------------------------------------

print("\n--- Part 3: Generating Visualization Plots ---")
num_cols = len(meteo_vars)
meteo_list = list(meteo_vars.keys())

# --- Calculate Global Extent ---
reference_data_for_extent = ds_global_mean[meteo_list[0]]
global_extent = calculate_dynamic_extent(reference_data_for_extent)
if global_extent is None:
    print("ERROR: No valid data found for global extent calculation. Using default.")
    global_extent = [ds_global_mean.longitude.min().item(), ds_global_mean.longitude.max().item(), 
                     ds_global_mean.latitude.min().item(), ds_global_mean.latitude.max().item()]
    
# --- Define Font Sizes (UNCHANGED) ---
TITLE_FONT_SIZE = 15
SUBTITLE_FONT_SIZE = 13
AXIS_FONT_SIZE = 11
CBAR_FONT_SIZE = 10


# --- PLOT SET 1: Global Mean Climatologies (UNCHANGED) ---
print("1. Plotting Global Mean Climatologies (Refined Style)...")

fig1, axes1 = plt.subplots(
    1, num_cols, 
    figsize=(4 * num_cols, 3.5),
    subplot_kw={'projection': ccrs.PlateCarree()} 
)

if num_cols == 1: axes1 = [axes1]

for i, var_name in enumerate(meteo_list):
    ax = axes1[i]
    
    mean_data = global_means[var_name]
    unit, cmap = meteo_vars[var_name]
    var_display_name = meteo_var_names[var_name]
    
    plot = mean_data.plot.pcolormesh(
        ax=ax, 
        x='longitude', 
        y='latitude', 
        cmap=cmap, 
        transform=ccrs.PlateCarree(), 
        add_colorbar=False,
        rasterized=True
    )
    
    ax.set_extent(global_extent, crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linewidth=0.8)
    
    ax.set_title(f"{var_display_name}", fontsize=SUBTITLE_FONT_SIZE) 
    ax.set_xlabel("Longitude", fontsize=AXIS_FONT_SIZE)
    ax.set_ylabel("Latitude", fontsize=AXIS_FONT_SIZE)
    
    cbar = plt.colorbar(
        plot, 
        ax=ax, 
        orientation='horizontal', 
        pad=0.08, 
        aspect=40,
        label=f'({unit})',
        shrink=0.7
    )
    cbar.locator = MaxNLocator(nbins=8)
    cbar.update_ticks()
    cbar.ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    cbar.set_label(f"({unit})", fontsize=CBAR_FONT_SIZE)
    cbar.ax.tick_params(labelsize=CBAR_FONT_SIZE)

fig1.suptitle("Global Mean Climatology of Meteorological Variables", fontsize=TITLE_FONT_SIZE, y=1.02)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()
print("Global Mean plots displayed.")

# --- PLOT SET 2: Raw Composites (DEDICATED COLORBARS) ---
print("\n2. Plotting Raw Composites (Individual 2-Panel Figures with Dedicated Colorbars)...")

for pollutant in pollutant_files.keys():
    if pollutant not in all_composite_results: continue
    
    for var_name in meteo_list:
        result = all_composite_results[pollutant][var_name]
        high_comp = result['High_Comp']
        low_comp = result['Low_Comp']
        unit, cmap = result['unit'], meteo_vars[var_name][1]
        var_display_name = meteo_var_names[var_name]
        
        # Calculate consistent color scale for the two panels
        max_val = np.nanmax([high_comp.values, low_comp.values])
        min_val = np.nanmin([high_comp.values, low_comp.values])
        
        # Create a new 1x2 figure - Increased size to fit two vertical colorbars
        fig, axes = plt.subplots(
            1, 2, figsize=(10, 4), # Increased width
            subplot_kw={'projection': ccrs.PlateCarree()}
        )
        
        # --- High Composite Plot (Left Panel) ---
        ax_high = axes[0]
        plot_high = high_comp.plot.pcolormesh(
            ax=ax_high, x='longitude', y='latitude', cmap=cmap, 
            vmin=min_val, vmax=max_val, add_colorbar=False, # Colorbar handled manually
            transform=ccrs.PlateCarree(), rasterized=True
        )
        ax_high.set_extent(global_extent, crs=ccrs.PlateCarree())
        ax_high.coastlines()
        ax_high.add_feature(cfeature.BORDERS, linewidth=0.8)
        ax_high.set_title(f"High {pollutant} Composite", fontsize=SUBTITLE_FONT_SIZE)
        ax_high.set_xlabel("Longitude", fontsize=AXIS_FONT_SIZE)
        ax_high.set_ylabel("Latitude", fontsize=AXIS_FONT_SIZE)
        
        # Dedicated Colorbar for High Plot
        cbar_high = plt.colorbar(
            plot_high, 
            ax=ax_high, 
            orientation='horizontal', 
            shrink=0.8, 
            pad=0.05
        )
        cbar_high.set_label(f'({unit})', fontsize=CBAR_FONT_SIZE)

        # --- Low Composite Plot (Right Panel) ---
        ax_low = axes[1]
        plot_low = low_comp.plot.pcolormesh(
            ax=ax_low, x='longitude', y='latitude', cmap=cmap, 
            vmin=min_val, vmax=max_val, add_colorbar=False, # Colorbar handled manually
            transform=ccrs.PlateCarree(), rasterized=True
        )
        ax_low.set_extent(global_extent, crs=ccrs.PlateCarree())
        ax_low.coastlines()
        ax_low.add_feature(cfeature.BORDERS, linewidth=0.8)
        ax_low.set_title(f"Low {pollutant} Composite", fontsize=SUBTITLE_FONT_SIZE)
        ax_low.set_xlabel("Longitude", fontsize=AXIS_FONT_SIZE)
        ax_low.set_ylabel("") # Suppress repeated Y-label
        
        # Dedicated Colorbar for Low Plot
        cbar_low = plt.colorbar(
            plot_low, 
            ax=ax_low, 
            orientation='horizontal', 
            shrink=0.8, 
            pad=0.05
        )
        cbar_low.set_label(f'({unit})', fontsize=CBAR_FONT_SIZE)


        fig.suptitle(f"{var_display_name} based on {pollutant} Extremes (n={N_days})", fontsize=TITLE_FONT_SIZE, y=1.02)
        plt.tight_layout(rect=[0, 0, 1, 0.98]) # Adjusted to accommodate two vertical colorbars
        plt.show()
        print(f"  -> Raw Composite plot for {pollutant} and {var_name} displayed with dedicated colorbars.")


# --- PLOT SET 3: Composite Anomalies (DEDICATED COLORBARS) ---
print("\n3. Plotting Composite Anomalies (Refined Individual 2-Panel Figures with Dedicated Colorbars)...")

for pollutant in pollutant_files.keys():
    if pollutant not in all_composite_results: continue
    
    for var_name in meteo_list:
        result = all_composite_results[pollutant][var_name]
        high_anomaly = result['High_Anomaly']
        low_anomaly = result['Low_Anomaly']
        unit = result['unit']
        var_display_name = meteo_var_names[var_name]
        
        # Calculate consistent diverging color scale
        max_abs = np.nanmax(np.abs([high_anomaly.values, low_anomaly.values])) if high_anomaly.values.size > 0 else 1e-6
        
        # Create a new 1x2 figure - Increased size to fit two vertical colorbars
        fig, axes = plt.subplots(
            1, 2, figsize=(10, 4), # Increased width
            subplot_kw={'projection': ccrs.PlateCarree()}
        )
        
        # --- High Anomaly Plot (Left Panel) ---
        ax_high = axes[0]
        plot_high = high_anomaly.plot.pcolormesh(
            ax=ax_high, x='longitude', y='latitude', cmap='RdBu_r', 
            vmin=-max_abs, vmax=max_abs, add_colorbar=False, # Colorbar handled manually
            transform=ccrs.PlateCarree(), rasterized=True
        )
        ax_high.set_extent(global_extent, crs=ccrs.PlateCarree())
        ax_high.coastlines()
        ax_high.add_feature(cfeature.BORDERS, linewidth=0.8)
        ax_high.set_title(f"High {pollutant} Anomaly", fontsize=SUBTITLE_FONT_SIZE)
        ax_high.set_xlabel("Longitude", fontsize=AXIS_FONT_SIZE)
        ax_high.set_ylabel("Latitude", fontsize=AXIS_FONT_SIZE)
        
        # Dedicated Vertical Colorbar for High Plot
        cbar_high = plt.colorbar(
            plot_high, 
            ax=ax_high, 
            orientation='horizontal', 
            shrink=0.8, 
            pad=0.05
        )
        cbar_high.set_label(f'({unit})', fontsize=CBAR_FONT_SIZE)


        # --- Low Anomaly Plot (Right Panel) ---
        ax_low = axes[1]
        plot_low = low_anomaly.plot.pcolormesh(
            ax=ax_low, x='longitude', y='latitude', cmap='RdBu_r', 
            vmin=-max_abs, vmax=max_abs, add_colorbar=False, # Colorbar handled manually
            transform=ccrs.PlateCarree(), rasterized=True
        )
        ax_low.set_extent(global_extent, crs=ccrs.PlateCarree())
        ax_low.coastlines()
        ax_low.add_feature(cfeature.BORDERS, linewidth=0.8)
        ax_low.set_title(f"Low {pollutant} Anomaly", fontsize=SUBTITLE_FONT_SIZE)
        ax_low.set_xlabel("Longitude", fontsize=AXIS_FONT_SIZE)
        ax_low.set_ylabel("")
        
        # Dedicated Vertical Colorbar for Low Plot
        cbar_low = plt.colorbar(
            plot_low, 
            ax=ax_low, 
            orientation='horizontal', 
            shrink=0.8, 
            pad=0.05
        )
        cbar_low.set_label(f'({unit})', fontsize=CBAR_FONT_SIZE)


        fig.suptitle(f"{var_display_name} based on {pollutant} Extremes (n={N_days})", fontsize=TITLE_FONT_SIZE, y=1.02)
        plt.tight_layout(rect=[0, 0, 1, 0.98])
        plt.show()
        print(f"  -> Anomaly plot for {pollutant} and {var_name} displayed with dedicated colorbars.")

print("\nCode execution finished. Composites and Anomalies now have dedicated vertical colorbars for each high and low panel.")